In [ ]:
import pandas as pd
import numpy as np
import datetime
import os
from tqdm import tqdm # To display the progress bar
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

import warnings
warnings.filterwarnings("ignore")

In [ ]:
def arrange(path_file):
  # Uploading the data
  data = pd.read_excel(path_file)
  data_provider = data["Unnamed: 2"].loc[0]
  data_notes = data["Unnamed: 1"].loc[data.shape[0]-1]
  param = path_file.split("/")[-1].split("-")[1].split(".")[0]
  data = data.drop(data[data["Unnamed: 2"] == data_provider].index) # T.C. Çevre Şehircilik çıkardım
  data = data.drop(data[data["Unnamed: 1"] == data_notes].index) # En alttaki notları çıkardım
  data = data.drop(data.tail(2).index) # Notlarla ilgili gereksiz son 2 satırı çıkardım
  data = data.dropna(how="all") # Bütün satır NA ise çıkardım
  data = data.dropna(how="all", axis = 1) # Bütün sütun NA ise çıkardım

  # Creating a table for each station and each year
  justStarted = True
  valuelst = [] # her ay için ayrı liste
  dflst = []
  for i, row in data.iterrows(): # data içindeki her satır
    if "Yıl" in str(row[0]):
      if not(justStarted):
        valueFlatlst = valuelst
        df = pd.DataFrame({"year": year, "station": station, "stationID": stationID,
                          param: valueFlatlst})
        dflst.append(df)
        valuelst = []
      else:
        justStarted = False
      year = int(row[0].split()[1])
      station = row[0].split()[-1][:-6]
      stationID = row[0].split()[-1][-5:]

    elif "Gün" in str(row[0]) or param in str(row[0]):
      continue

    else:
      for j in range(12):
        valuelst.append(float(row.drop("Unnamed: 1")[j]))

    # Creating datetime for each data
  for df in dflst:
    datetimelst = []
    isReallst = []
    for m in range(1,13):
      for d in range(1,32):
        try:
          datetimelst.append(datetime.datetime(df.year[0], m, d))
          isReallst.append(1)
        except ValueError: # if day is out of range for month (including artık yıl)
          datetimelst.append(np.nan) # because the impossible dates has a NA values
          # like: 30/02/2021 --> NA
          isReallst.append(0)

    df["date"] = datetimelst
    df["isReal"] = isReallst

  # Eliminating data who are fake (like 30/02/2022)
  for i in range(len(dflst)):
    dflst[i] = dflst[i][dflst[i].isReal == 1]

  return pd.concat(dflst), param, data_provider, data_notes

In [ ]:
path = "/content/drive/MyDrive/PROJ201/Meteorological_Data/"
datadict = {}
for file in (os.listdir(path)):
  print(file)
  path_file = path+file
  data, param, _, _ = tqdm(arrange(path_file))
  datadict[param] = data

2023112319BE-Günlük Toplam Buharlaşma Evapotranspirasyon (mm).xlsx


100%|██████████| 4/4 [00:00<00:00, 3144.16it/s]


2023112319BE-Günlük Ortalama Sıcaklık (°C).xlsx


100%|██████████| 4/4 [00:00<00:00, 34952.53it/s]


2023112319BE-Günlük Minimum Sıcaklık (C).xlsx


...

100%|██████████| 4/4 [00:00<00:00, 38926.26it/s]


2023112319BE-Günlük Ortalama Nispi Nem (%).xlsx


100%|██████████| 4/4 [00:00<00:00, 37957.50it/s]


2023112319BE-Günlük Maksimum Sıcaklık (°C).xlsx


100%|██████████| 4/4 [00:00<00:00, 31300.78it/s]


In [ ]:
maindf = list(datadict.values())[0]
for df in list(datadict.values()):
  maindf = pd.merge(maindf, df, how = "outer")

In [ ]:
maindf

,year,station,stationID,Günlük Toplam Buharlaşma Evapotranspirasyon (mm),date,isReal,Günlük Ortalama Sıcaklık (°C),Günlük Minimum Sıcaklık (C),Günlük Toplam Yağış (mm=kg÷m²) MANUEL,Günlük Toplam Yağış (mm=kg÷m²) OMGİ,Günlük Ortalama Nispi Nem (%),Günlük Maksimum Sıcaklık (°C)
0,2021,MARMARA,17814,6.6,2021-01-01,1,13.0,10.4,NaN,4.4,75.5,15.2
1,2021,MARMARA,17814,0.0,2021-01-02,1,14.0,10.7,NaN,0.0,57.5,17.8
2,2021,MARMARA,17814,2.3,2021-01-03,1,6.6,3.4,NaN,0.0,73.9,11.9
3,2021,MARMARA,17814,4.4,2021-01-04,1,7.0,4.2,NaN,0.0,69.1,11.2
4,2021,MARMARA,17814,7.3,2021-01-05,1,21.7,16.4,NaN,0.0,57.0,28.6
...,...,...,...,...,...,...,...,...,...,...,...,...
27370,2022,ÜMRANİYE,18403,NaN,2022-12-27,1,23.9,21.4,NaN,0.2,87.6,27.5
27371,2022,ÜMRANİYE,18403,NaN,2022-12-28,1,NaN,NaN,NaN,NaN,NaN,NaN
27372,2022,ÜMRANİYE,18403,NaN,2022-12-29,1,13.6,10.0,NaN,0.0,81.7,19.7
27373,2022,ÜMRANİYE,18403,NaN,2022-12-30,1,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
main2df = maindf.set_index(['date', 'stationID'])
main2df = main2df.drop(columns=["isReal", "year"])
main2df = main2df.rename(columns = {"Günlük Toplam Buharlaşma Evapotranspirasyon (mm)":"evapo",
                                    "Günlük Ortalama Sıcaklık (°C)": "mean_temp",
                                    "Günlük Minimum Sıcaklık (C)": "min_temp",
                                    "Günlük Toplam Yağış (mm=kg÷m²) MANUEL": "total_precip_manuel",
                                    "Günlük Toplam Yağış (mm=kg÷m²) OMGİ ": "total_precip_omgi",
                                    "Günlük Ortalama Nispi Nem (%)": "rel_hum",
                                    "Günlük Maksimum Sıcaklık (°C)":"max_temp"})
main2df

,,station,evapo,mean_temp,min_temp,total_precip_manuel,total_precip_omgi,rel_hum,max_temp
date,stationID,,,,,,,,
2021-01-01,17814,MARMARA,6.6,13.0,10.4,NaN,4.4,75.5,15.2
2021-01-02,17814,MARMARA,0.0,14.0,10.7,NaN,0.0,57.5,17.8
2021-01-03,17814,MARMARA,2.3,6.6,3.4,NaN,0.0,73.9,11.9
2021-01-04,17814,MARMARA,4.4,7.0,4.2,NaN,0.0,69.1,11.2
2021-01-05,17814,MARMARA,7.3,21.7,16.4,NaN,0.0,57.0,28.6
...,...,...,...,...,...,...,...,...,...
2022-12-27,18403,ÜMRANİYE,NaN,23.9,21.4,NaN,0.2,87.6,27.5
2022-12-28,18403,ÜMRANİYE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-29,18403,ÜMRANİYE,NaN,13.6,10.0,NaN,0.0,81.7,19.7


In [ ]:
main2df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 27375 entries, (Timestamp('2021-01-01 00:00:00'), '17814') to (Timestamp('2022-12-31 00:00:00'), '18403')
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   station              27375 non-null  object 
 1   evapo                3689 non-null   float64
 2   mean_temp            23595 non-null  float64
 3   min_temp             23595 non-null  float64
 4   total_precip_manuel  1467 non-null   float64
 5   total_precip_omgi    22452 non-null  float64
 6   rel_hum              23339 non-null  float64
 7   max_temp             23594 non-null  float64
dtypes: float64(7), object(1)
memory usage: 1.8+ MB


In [ ]:
describe_table = main2df.describe().T
main2df_nanlst = []
for param in main2df.columns:
  main2df_nanlst.append(main2df[param].isnull().sum() / main2df.size *100)
describe_table["nan_values_perc"] = main2df_nanlst[1:] # to remove for stationID

In [ ]:
describe_table

,count,mean,std,min,25%,50%,75%,max,nan_values_perc
evapo,3689.0,8.003768,8.292770,0.0,3.2,5.7,9.5,51.4,10.815525
mean_temp,23595.0,15.467578,7.304937,-5.3,9.9,15.4,21.9,33.1,1.726027
min_temp,23595.0,11.986879,7.039023,-10.8,6.8,11.9,17.8,31.6,1.726027
total_precip_manuel,1467.0,4.450170,6.890974,0.0,0.3,1.7,5.6,74.0,11.830137
total_precip_omgi,22452.0,1.723855,5.309187,0.0,0.0,0.0,0.5,221.4,2.247945
rel_hum,23339.0,76.059626,12.397266,30.0,67.8,76.1,85.0,100.0,1.842922
max_temp,23594.0,19.748623,8.035725,-1.4,13.7,19.9,26.6,44.1,1.726484


In [ ]:
main2df.to_csv("/content/drive/MyDrive/PROJ201/Arranged_Compact_Air_Data/Compact_Meteorological_Data.csv")
describe_table.to_csv("/content/drive/MyDrive/PROJ201/Statistical Graphs/DescribeTableMeteorological.csv")
main2df.groupby("station").describe().T.to_csv("/content/drive/MyDrive/PROJ201/Statistical Graphs/DescribeTableMeteorologicalPerStation.csv")